In [1]:
#!pip install keras_tuner
#!pip install tensorflow
#!pip install yfinance
#!pip install pandas_ta
#!pip install plotly
#!pip install numpy==1.20.3
#!pip install scikit-learn
#!pip install pyarrow
#!pip install fastparquet

In [2]:
print("Importa librerie", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
import gc
print(".", end="", flush=True)
gc.enable()
import os
print("=", end="\n", flush=True)
print("ok")

Importa librerieUsing TensorFlow backend


: 

In [ ]:
n_timesteps = 120 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 5 # n. barre nel futuro di cui si desidera prevedere il prezzo
set_file_x_y = "_tutti"
initial_lr = 0.001

features_prezzo = [
    "Close",
#    "EMA_5", 
#    "EMA_20", 
#    "EMA_50",
    "Open",  
    "High",
    "Low",
#    "PSAR",
#    "SUPERT", 
]

features_da_scalare_singolarmente = [
    "Volume",
#    "ATR",
#    "PSARaf",
#    "ADX",
]

features_meno_piu = [
#    "MACDh",    
#    "AROONOSC",
#    "TRIX",
#    "TRIXs",
#    "DM_OSC",
]

features_no_scala = [
#    "SUPERTd",  
#    "PSARr"
]

features_candele = [
#    "CDL_2CROWS", "CDL_3BLACKCROWS", "CDL_3INSIDE", "CDL_3LINESTRIKE", "CDL_3OUTSIDE", "CDL_3STARSINSOUTH", "CDL_3WHITESOLDIERS", "CDL_ABANDONEDBABY", "CDL_ADVANCEBLOCK", "CDL_BELTHOLD", "CDL_BREAKAWAY", "CDL_CLOSINGMARUBOZU", "CDL_CONCEALBABYSWALL", "CDL_COUNTERATTACK", "CDL_DARKCLOUDCOVER", "CDL_DOJI_10_0.1", "CDL_DOJISTAR", "CDL_DRAGONFLYDOJI", "CDL_ENGULFING", "CDL_EVENINGDOJISTAR", "CDL_EVENINGSTAR", "CDL_GAPSIDESIDEWHITE", "CDL_GRAVESTONEDOJI", "CDL_HAMMER", "CDL_HANGINGMAN", "CDL_HARAMI", "CDL_HARAMICROSS", "CDL_HIGHWAVE", "CDL_HIKKAKE", "CDL_HIKKAKEMOD", "CDL_HOMINGPIGEON", "CDL_IDENTICAL3CROWS", "CDL_INNECK", "CDL_INSIDE", "CDL_INVERTEDHAMMER", "CDL_KICKING", "CDL_KICKINGBYLENGTH", "CDL_LADDERBOTTOM", "CDL_LONGLEGGEDDOJI", "CDL_LONGLINE", "CDL_MARUBOZU", "CDL_MATCHINGLOW", "CDL_MATHOLD", "CDL_MORNINGDOJISTAR", "CDL_MORNINGSTAR", "CDL_ONNECK", "CDL_PIERCING", "CDL_RICKSHAWMAN", "CDL_RISEFALL3METHODS", "CDL_SEPARATINGLINES", "CDL_SHOOTINGSTAR", "CDL_SHORTLINE", "CDL_SPINNINGTOP", "CDL_STALLEDPATTERN", "CDL_STICKSANDWICH", "CDL_TAKURI", "CDL_TASUKIGAP", "CDL_THRUSTING", "CDL_TRISTAR", "CDL_UNIQUE3RIVER", "CDL_UPSIDEGAP2CROWS", "CDL_XSIDEGAP3METHODS",
]

elenco_targets = [
#    "EMA_5",
#    "EMA_20", 
#    "EMA_50",
    #"Open",
    #"High",
    #"Low",
    "Close"
]

col_features_prezzo = {col: idx for idx, col in enumerate(features_prezzo)}
col_features_da_scalare_singolarmente = {col: idx for idx, col in enumerate(features_da_scalare_singolarmente)}
col_features_meno_piu = {col: idx for idx, col in enumerate(features_meno_piu)}
col_features_no_scala = {col: idx for idx, col in enumerate(features_no_scala)}
col_features_candele = {col: idx for idx, col in enumerate(features_candele)}
col_targets = {col: idx for idx, col in enumerate(elenco_targets)}
n_features = len(col_features_prezzo) + len(col_features_da_scalare_singolarmente) + len(col_features_meno_piu) + len(col_features_no_scala) + len(col_features_candele) 
n_targets = len(col_targets) 


print("ok")


In [ ]:
if os.path.exists("lista_ticker.parquet"):
    print("Caricamento lista_ticker esistente", flush=True)
    lista_ticker = pd.read_parquet("lista_ticker.parquet")
else:
    print("Download lista ticker", flush=True)
    lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
    lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
    lista_ticker = lista_ticker.loc[(lista_ticker["Categoria"] != "D"), :]
    lista_ticker.to_parquet("lista_ticker.parquet")

n_simboli_addestramento = len(lista_ticker)

if os.path.exists("indice.npy"):    
    inizio = np.load("indice.npy")
    inizio += 1
else:
    inizio = 0
print("ok")

In [ ]:
if os.path.exists(f"dati/X{set_file_x_y}.npy") and os.path.exists(f"dati/Y{set_file_x_y}.npy"):
    print("Caricamento X e Y")
    X = np.load(f'dati/X{set_file_x_y}.npy')
    Y = np.load(f'dati/Y{set_file_x_y}.npy')
else:
    X = np.zeros((0, n_timesteps, n_features))
    Y = np.zeros((0, giorni_previsione, n_targets))
    lista_x, lista_y = [], []
    for i_ticker in range (inizio, n_simboli_addestramento):
        nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
        print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
        print("Download dati ticker", flush=True)
        try:
            ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
            if ticker["Close"].iloc[-1] >= 1:
                ticker.index = ticker.index.date
                print("Calcolo indicatori ticker", flush=True)
                ticker = fx.crea_indicatori(ticker)
                ticker.dropna(axis=0, inplace=True)

                print("Definizione features e target", flush=True)
                idx, X_train, Y_train, _ = fx.to_XY(ticker, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

                print("Aggiunta dati a liste X e Y", flush=True)
                lista_x.append(X_train)        
                lista_y.append(Y_train)   
                if (i_ticker % 100 == 0) and (i_ticker > 0):
                    print("Concatenamento su X, Y", flush=True)
                    X_arr = np.vstack(lista_x)
                    Y_arr = np.vstack(lista_y)
                    X = np.vstack((X, X_arr))
                    Y = np.vstack((Y, Y_arr))
                    print("Salvataggio X e Y su file", flush=True)
                    np.save(f'dati/X{set_file_x_y}', X)
                    np.save(f'dati/Y{set_file_x_y}', Y)
                    np.save("indice", i_ticker)
                    lista_x, lista_y = [], []
                gc.collect()
        except Exception as e:
            print(e)
            gc.collect()
            continue
            
    print("Concatenamento su X, Y", flush=True)
    X_arr = np.vstack(lista_x)
    Y_arr = np.vstack(lista_y)
    X = np.vstack((X, X_arr))
    Y = np.vstack((Y, Y_arr))
    print("Salvataggio X e Y su file", flush=True)
    np.save(f'dati/X{set_file_x_y}', X)
    np.save(f'dati/Y{set_file_x_y}', Y)
    np.save("indice", i_ticker)
    lista_x, lista_y = [], []

    os.remove("indice.npy")
    os.remove("lista_ticker.parquet")
    gc.collect()
print("ok")